In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession



config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

root = os.getcwd()


In [2]:
def data_generator(batch_size):
    
    path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
    path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
    start_index = 0
    while True:
        if start_index == 1:
            break
        encoded_images_array = []
        ground_truth_array = []
    
        folders = os.listdir(path_post_enc)
        for i in range(start_index, start_index + batch_size):
            os.chdir(os.path.join(path_post_enc, folders[i]))
            files = os.listdir(os.getcwd())
            for file in files:
                with open(file, 'rb') as yuv_file:
                    encoded_images_array.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))
                    
        folders = os.listdir(path_pre_enc)
        for i in range(start_index, start_index + batch_size):
            os.chdir(os.path.join(path_pre_enc, folders[i]))
            files = os.listdir(os.getcwd())
            for file in files:
                with open(file, 'rb') as yuv_file:
                    ground_truth_array.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))
        start_index += batch_size
        
        encoded_images_array = np.array(encoded_images_array) / 255.0
        ground_truth_array = np.array(ground_truth_array) / 255.0

        yield(encoded_images_array, ground_truth_array)


In [3]:
def validation_data(X_val, y_val):
    path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
    path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
    
    folders = os.listdir(path_post_enc)
    for i in range(570, 578):
        os.chdir(os.path.join(path_post_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                X_val.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

    folders = os.listdir(path_pre_enc)
    for i in range(570, 578):
        os.chdir(os.path.join(path_pre_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                y_val.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

In [4]:

def SpatialAttention(input):
    x = layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same')(input)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)
    out = layers.Activation('sigmoid')(x)

    return out
    
'''
def SpatialAttention(input):
    x_shortcut = input
    x = layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same')(input)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('sigmoid')(x)
    out = layers.Multiply()([x_shortcut, x])

    return out
    '''

"\ndef SpatialAttention(input):\n    x_shortcut = input\n    x = layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same')(input)\n    x = layers.ReLU()(x)\n    x = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)\n    x = layers.Activation('sigmoid')(x)\n    out = layers.Multiply()([x_shortcut, x])\n\n    return out\n    "

In [5]:

def ChannelAttention(input):
    sh = input.get_shape().as_list()
    x = layers.GlobalAveragePooling2D()(input)
    x = layers.Dense(8)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(sh[-1])(x)
    out = layers.Activation('sigmoid')(x)

    return out


In [6]:
def ResidualAttentionBlock(x):
    x_shortcut = x

    x = layers.Conv2D(filters=1, kernel_size=3, padding="same")(x)  # conv 1
   # x = layers.BatchNormalization()(x)
    spatial_attention = SpatialAttention(x)  # spatial attention
    x = layers.Multiply()([x, spatial_attention])  # multiply

    x = layers.Conv2D(filters=16, kernel_size=3, padding="same")(x)  # conv 2
   # x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.05)(x)  # leaky relu
    x = layers.Conv2D(filters=1, kernel_size=3, padding="same")(x)  # conv 2
  #  x = layers.BatchNormalization()(x)

    x = layers.Add()([x, x_shortcut])  # add
  #  x = layers.BatchNormalization()(x)
  #  x = activations.sigmoid(x)

    return x


In [7]:
def CNN(num_blocks):
    input_layer = layers.Input(shape=(256, 256, 1))
    x = input_layer

    for _ in range(num_blocks):
        x = ResidualAttentionBlock(x)

    model = tf.keras.Model(inputs=input_layer, outputs=x)
    return model


In [8]:
num_blocks = 30

optimizer = optimizers.Adam(learning_rate=0.001)

Model = CNN(num_blocks)

Model.compile(optimizer=optimizer, loss='mae')
# Model.summary()

In [9]:
batch_size = 2
data_gen = data_generator(batch_size)

In [10]:

Model.fit(
    x=data_gen,
    steps_per_epoch=1,
    shuffle=False,
)



ResourceExhaustedError: Graph execution error:

Detected at node 'model/leaky_re_lu/LeakyRelu' defined at (most recent call last):
    File "C:\Users\Furkan\anaconda3\envs\tensorflowgpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Furkan\anaconda3\envs\tensorflowgpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Furkan\anaconda3\envs\tensorflowgpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Furkan\anaconda3\envs\tensorflowgpu\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Furkan\anaconda3\envs\tensorflowgpu\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\Users\Furkan\anaconda3\envs\tensorflowgpu\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.r

In [ ]:
predictions = Model.predict(X_val)

In [ ]:
predictions[predictions < 0] = 0

predictions[predictions > 255] = 255

In [ ]:
os.chdir(root)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\as.png", predictions[0] * 255)